<a href="https://colab.research.google.com/github/NeuromatchAcademy/course-content-dl/blob/main/projects/ComputerVision/image_alignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Alignment 

**By Neuromatch Academy**

__Content creators:__ Kaleb Vinehout

__Production editor:__ Spiros Chavlis

**Our 2021 Sponsors, including Presenting Sponsor Facebook Reality Labs**

<p align='center'><img src='https://github.com/NeuromatchAcademy/widgets/blob/master/sponsors.png?raw=True'/></p>

---
# Objective

This notebook will give you starting points to perform Spatial Transformers. These can be used for registraion of images. This is useful when comparing multiple datasets together. Check out https://arxiv.org/abs/1506.02025 for more details.These can also be used to plug into any CNN architecture to deal with dataset rotations and scale invariance in a given dataset

* Spatial transformers contain three main parts.The first is a localizaion net the second is grid generator and the last is a sampler. 

---
# Intro to Image Alignment
  

## Image Alignment Applications
* To answer many biological questions, it is necessary to align sets of images together
* Use Spatial Transfomers as a preprocessing step for any CNN achitecutre. This could be done before facial recognition in order to crop and align images before spatial recognition.

## Acknowledgments:
This Notebook was developed by Kaleb Vinehout. It borrows from material by Ghassen Hamrouni, Asror Wali, and Erwin Russel.

---
# Setup

In [ ]:
# @title Install dependencies
!pip install scikit-image --quiet
!pip install Pillow --quiet
!pip install n2v --quiet
!pip install csbdeep --quiet

In [ ]:
# Imports
import glob
import time
import sklearn.decomposition

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
from torchvision import transforms
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
from torchvision import datasets, transforms

from PIL import Image
from skimage.util import random_noise

In [ ]:
# @title Figure settings
%matplotlib inline
plt.ion()   # interactive mode

In [ ]:
# @title Set device (GPU or CPU). Execute `set_device()`
# especially if torch modules used.

# inform the user if the notebook uses GPU or CPU.

def set_device():
  device = "cuda" if torch.cuda.is_available() else "cpu"
  if device != "cuda":
    print("GPU is not enabled in this notebook. \n"
          "If you want to enable it, in the menu under `Runtime` -> \n"
          "`Hardware accelerator.` and select `GPU` from the dropdown menu")
  else:
    print("GPU is enabled in this notebook. \n"
          "If you want to disable it, in the menu under `Runtime` -> \n"
          "`Hardware accelerator.` and select `None` from the dropdown menu")

  return device

In [ ]:
device = set_device()

---
# Data loading

## Loader for classic MNIST as an example

In [ ]:
# @title Download MNIST dataset
import tarfile, requests, os

fname = 'MNIST.tar.gz'
name = 'MNIST'
url = 'https://osf.io/y2fj6/download'

if not os.path.exists(name):
  print('\nDownloading MNIST dataset...')
  r = requests.get(url, allow_redirects=True)
  with open(fname, 'wb') as fh:
    fh.write(r.content)
  print('\nDownloading MNIST completed.')

if not os.path.exists(name):
  with tarfile.open(fname) as tar:
    tar.extractall()
    os.remove(fname)
else:
  print('MNIST dataset is dowloaded.')

In [ ]:
# Training dataset
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='.', train=True, download=False,
                   transform=transforms.Compose([
                                                 transforms.ToTensor(),
                                                 transforms.Normalize((0.1307,), (0.3081,))
                                                 ])),
                                           batch_size=64,
                                           shuffle=True,
                                           num_workers=2)
# Test dataset
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='.', train=False, download=False,
                   transform=transforms.Compose([
                                                 transforms.ToTensor(),
                                                 transforms.Normalize((0.1307,), (0.3081,))
                                                 ])),
                                          batch_size=64,
                                          shuffle=True,
                                          num_workers=2)

Define functions to convert between Tensor and numpy image

In [ ]:
def convert_image_np(inp):
  """Convert a Tensor to numpy image."""
  inp = inp.numpy().transpose((1, 2, 0))
  mean = np.array([0.485, 0.456, 0.406])
  std = np.array([0.229, 0.224, 0.225])
  inp = std * inp + mean
  inp = np.clip(inp, 0, 1)
  return inp

In [ ]:
def convert2tensor(self, args):
  data = np.asarray([e[0] for e in self.binary_train_dataset])
  target = np.asarray([e[1] for e in self.binary_train_dataset])

  tensor_data = torch.from_numpy(data)
  tensor_data = tensor_data.float()
  tensor_target = torch.from_numpy(target)

  train = data_utils.TensorDataset(tensor_data, tensor_target)
  train_loader = data_utils.DataLoader(train, batch_size=args.batch_size, shuffle = True)
  return train_loader

Plot the data

In [ ]:
## Display Images
# Get a batch of training data
data = next(iter(test_loader))[0].to(device)
input_tensor = data.cpu()
in_grid = convert_image_np(torchvision.utils.make_grid(input_tensor))

# Plot the images
plt.figure()
plt.imshow(in_grid)
plt.show()

# plot ONE image
plt.figure()
plt.imshow(torchvision.utils.make_grid(input_tensor).numpy().transpose((1, 2, 0)))
plt.show()

---
# Spatial Transformer on images



## Spatial Transformer Network

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.conv2_drop = nn.Dropout2d()
    self.fc1 = nn.Linear(320, 50)
    self.fc2 = nn.Linear(50, 10)

    # Spatial transformer localization-network
    self.localization = nn.Sequential(
        nn.Conv2d(1, 8, kernel_size=7),
        nn.MaxPool2d(2, stride=2),
        nn.ReLU(True),
        nn.Conv2d(8, 10, kernel_size=5),
        nn.MaxPool2d(2, stride=2),
        nn.ReLU(True)
    )

    # Regressor for the 3 * 2 affine matrix
    self.fc_loc = nn.Sequential(
        nn.Linear(10 * 3 * 3, 32),
        nn.ReLU(True),
        nn.Linear(32, 3 * 2)
    )

    # Initialize the weights/bias with identity transformation
    self.fc_loc[2].weight.data.zero_()
    self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

  # Spatial transformer network forward function
  def stn(self, x):
    xs = self.localization(x)
    xs = xs.view(-1, 10 * 3 * 3)
    theta = self.fc_loc(xs)
    theta = theta.view(-1, 2, 3)

    grid = F.affine_grid(theta, x.size())
    x = F.grid_sample(x, grid)

    return x

  def forward(self, x):
    # transform the input
    x = self.stn(x)

    # Perform the usual forward pass
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
    x = x.view(-1, 320)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)
    return F.log_softmax(x, dim=1)

## Train and Test functions for the STN

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

### Train function

In [ ]:
def train(train_loader, optimizer, epoch, device):

  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)

    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 500 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))

### Test function

In [ ]:
def test(test_loader, device):

  with torch.no_grad():
    model.eval()
    test_loss, correct = 0, 0

    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)

      # sum up batch loss
      test_loss += F.nll_loss(output, target, size_average=False).item()
      # get the index of the max log-probability
      pred = output.max(1, keepdim=True)[1]
      correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'
          .format(test_loss, correct, len(test_loader.dataset),
                  100. * correct / len(test_loader.dataset)))

### Run Train and test the data 

In [ ]:
num_epochs = 20
for epoch in range(1, num_epochs + 1):
  train(train_loader, optimizer, epoch, device)
  test(test_loader, device)

### Visualize the results

In [ ]:
def visualize_stn():
  with torch.no_grad():
    # Get a batch of training data
    data = next(iter(test_loader))[0].to(device)

    input_tensor = data.cpu()
    transformed_input_tensor = model.stn(data).cpu()

    in_grid = convert_image_np(
        torchvision.utils.make_grid(input_tensor))

    out_grid = convert_image_np(
        torchvision.utils.make_grid(transformed_input_tensor))

    # Plot the results side-by-side
    f, axarr = plt.subplots(1, 2)
    axarr[0].imshow(in_grid)
    axarr[0].set_title('Dataset Images')

    axarr[1].imshow(out_grid)
    axarr[1].set_title('Transformed Images')
  return in_grid, out_grid


# Visualize the STN transformation on some input batch
[in_grid, out_grid] = visualize_stn()
plt.ioff()
plt.show()

---
# Use Sørensen–Dice coefficient (DSC) to calculate the simularity of images --> modify for your data

## Function to compare similarity of images 

Compare the similarity of two images with the the Sørensen–Dice coefficient. See details [here](https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient)

In [ ]:
def calc_dice(im1, im2):
  """
  This calculates the DICE between two images. The maximum DICE is 1, the minimum is Zero.
  Args:
  -	im1, im2: one of the imges to calcualte DICE coeffeicnet. Note image1.shape has to equal image2.shape
  Returns:
  -	dice: the dice coeffeicent
  """
  im1 = np.asarray(im1).astype(np.bool)
  im2 = np.asarray(im2).astype(np.bool)

  if im1.shape != im2.shape:
    raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

  # Compute Dice coefficient
  intersection = np.logical_and(im1, im2)

  dice = 2. * intersection.sum() / (im1.sum() + im2.sum())

  return dice

## Run Dice on individual images set

In [ ]:
# what do you need to do to calculate dice for your images?
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='.', train=True, download=False,
                   transform=transforms.Compose([transforms.ToTensor()])),
                   batch_size=64,
                   shuffle=True,
                   num_workers=2)

images, labels = next(iter(train_loader))

idx1, idx2 = 1, 25
img1, img2 = images[1], images[20]

plt.figure()
plt.subplot(121)
plt.imshow(img1.squeeze())
plt.axis('off')
plt.title(f'Label: {labels[1]}')
plt.subplot(122)
plt.imshow(img2.squeeze())
plt.axis('off')
plt.title(f'Label: {labels[20]}')
plt.tight_layout()
plt.show()


dice = calc_dice(img1, img2)
print("The Dice is {}".format(dice))

---
# Optional preprocessing steps --> modify for your data

## Add salt/pepper noise to the dataset

### Add noise class

In [ ]:
# are there other types of noise you can add? What effect do different types of noise have? EX: gasusain?

def salt_pepper_noise(trainloader):
  for data in trainloader:
    img, _ = data[0], data[1]
    s_and_p = torch.tensor(random_noise(img, mode='s&p',
                                        salt_vs_pepper=0.5,
                                        clip=True))
  return s_and_p

add noise to both train and test datasets

plot clean and noisy data

## Remove noise with noise2void

Check out: https://aswali.github.io/WNet/ and this paper:https://arxiv.org/abs/1711.08506

In [ ]:
# @title Import `noise2void` dependances
from n2v.models import N2VConfig, N2V
from csbdeep.utils import plot_history
from n2v.utils.n2v_utils import manipulate_val_data
from n2v.internals.N2V_DataGenerator import N2V_DataGenerator

In [ ]:
# @title Make denoising function
def noise2void(data, model_name, patch_size=64):
  """
  Removes noise in 3d image using the noise 2 void method. Based on https://arxiv.org/abs/1811.10980 w/ this implementation: https://github.com/juglab/n2v
      Args:
      -	data: Numpy array  2d to be deionised
      -   model: name of model to load, if provided this model is used to denoise instead of model made from data otherwise this is name given to model (ex:#model_name = 'n2v_3D_blk')
      -patch_size: this is the size of patches in X and Y, default is 64
      Returns:
      -	data_denoise: Numpy array noise removed
  """
  # We create our DataGenerator-object.
  datagen = N2V_DataGenerator()
  # In the 'dims' parameter we specify the order of dimensions in the image files we are reading.
  if data.ndim == 2:
    print('2D image found to denosie')
    dataZYX = data
    data_exp = np.expand_dims(dataZYX, axis=(0, 1, 4))  # expand dimensions One at the front is used to hold a potential stack of images such as a movie, One at the end could hold color channels such as RGB. #expand dimensions One at the front is used to hold a potential stack of images such as a movie, One at the end could hold color channels such as RGB.
    patch_shape = (patch_size, patch_size)
    model_axis = 'YX'
  print('arrary with extra dimensions is size of {}'.format(data_exp.shape))
  print('patches are {}'.format(patch_shape))
  # the base directory in which our model will live
  basedir = 'models'
  path = basedir + '/' + model_name
  if not os.path.exists(path):
    print(path)
    # create model
    patches = datagen.generate_patches_from_list(data_exp, shape=patch_shape)
    print('patches shape {}'.format(patches.shape))
    # Patches are created so they do not overlap.
    # (Note: this is not the case if you specify a number of patches. See the docstring for details!)
    # Non-overlapping patches enable us to split them into a training and validation set.
    # modify split so set as a %
    perc_95 = int(patches.shape[0] * 0.95)
    X = patches[:perc_95]  # this is 600/640
    X_val = patches[perc_95:]  # this is 40/640

    # train model
    # You can increase "train_steps_per_epoch" to get even better results at the price of longer computation.
    fast = 128  # default
    slow = 50  # to get better results?  --> apply same model to Z plane
    speed = fast
    config = N2VConfig(X, unet_kern_size=3, train_steps_per_epoch=int(X.shape[0] / speed), train_epochs=20,
                        train_loss='mse', batch_norm=True, train_batch_size=4, n2v_perc_pix=0.198,
                        n2v_patch_shape=patch_shape, n2v_manipulator='uniform_withCP', n2v_neighborhood_radius=5)

    # Let's look at the parameters stored in the config-object.
    vars(config)
    # We are now creating our network model.
    model = N2V(config=config, name=model_name, basedir=basedir)
    history = model.train(X, X_val)
    print(sorted(list(history.history.keys())))
    model.export_TF(name='Noise2Void - data',
                    description='This is the 3D Noise2Void for  data.',
                    authors=["Kaleb Vinehout"],
                    test_img=X_val[0, ..., 0], axes=model_axis,
                    patch_shape=patch_shape)
  # run prediction model on rest of data in 3D image
  # A previously trained model is loaded by creating a new N2V-object without providing a 'config'.
  model = N2V(config=None, name=model_name, basedir=basedir)
  # Here we process the data.
  # The 'n_tiles' parameter can be used if images are too big for the GPU memory.
  # If we do not provide the 'n_tiles' parameter the system will automatically try to find an appropriate tiling.
  data_denoise = model.predict(dataZYX, axes=model_axis)  # , n_tiles=(2, 4, 4))

  return data_denoise

In [ ]:
# Runun the denoise on our data
data_denoise = noise2void(data, model_name='test_model', patch_size=64)

Make figures of raw data and denoised data

In [ ]:
# Make figures of raw data and denoised data
plt.figure()
plt.imshow(data, cmap='magma', vmin=np.percentile(data, 0.1), vmax=np.percentile(data, 99.9))
plt.show()

plt.figure()
plt.imshow(data_denoise, cmap='magma', vmin=np.percentile(data_denoise, 0.1), vmax=np.percentile(data_denoise, 99.9))
plt.show()

## Segment image with W-net

In [ ]:
# @title Wnet class
class Block(nn.Module):
  def __init__(self, in_filters, out_filters, seperable=True):
    super(Block, self).__init__()
    
    if seperable:
      self.spatial1=nn.Conv2d(in_filters, in_filters, kernel_size=3, groups=in_filters, padding=1)
      self.depth1=nn.Conv2d(in_filters, out_filters, kernel_size=1)
      self.conv1=lambda x: self.depth1(self.spatial1(x))
      self.spatial2=nn.Conv2d(out_filters, out_filters, kernel_size=3, padding=1, groups=out_filters)
      self.depth2=nn.Conv2d(out_filters, out_filters, kernel_size=1)
      self.conv2=lambda x: self.depth2(self.spatial2(x))

    else:
      self.conv1=nn.Conv2d(in_filters, out_filters, kernel_size=3, padding=1)
      self.conv2=nn.Conv2d(out_filters, out_filters, kernel_size=3, padding=1)
    
    
    self.relu1 = nn.ReLU()
    self.dropout1 = nn.Dropout(0.65) 
    self.batchnorm1=nn.BatchNorm2d(out_filters)

    self.relu2 = nn.ReLU()
    self.dropout2 = nn.Dropout(0.65) 
    self.batchnorm2=nn.BatchNorm2d(out_filters)

  def forward(self, x):
    x = self.batchnorm1(self.conv1(x)).clamp(0)
    x = self.relu1(x)
    x = self.dropout1(x)
    x = self.batchnorm2(self.conv2(x)).clamp(0)
    x = self.relu2(x)
    x = self.dropout2(x)
    
    return x


class UEnc(nn.Module):
  def __init__(self, squeeze, ch_mul=64, in_chans=3):
    super(UEnc, self).__init__()

    self.enc1=Block(in_chans, ch_mul, seperable=False)
    self.enc2=Block(ch_mul, 2*ch_mul)
    self.enc3=Block(2*ch_mul, 4*ch_mul)
    self.enc4=Block(4*ch_mul, 8*ch_mul)

    self.middle=Block(8*ch_mul, 16*ch_mul)

    self.up1=nn.ConvTranspose2d(16*ch_mul, 8*ch_mul, kernel_size=3, stride=2, padding=1, output_padding=1)
    self.dec1=Block(16*ch_mul, 8*ch_mul)
    self.up2=nn.ConvTranspose2d(8*ch_mul, 4*ch_mul, kernel_size=3, stride=2, padding=1, output_padding=1)
    self.dec2=Block(8*ch_mul, 4*ch_mul)
    self.up3=nn.ConvTranspose2d(4*ch_mul, 2*ch_mul, kernel_size=3, stride=2, padding=1, output_padding=1)
    self.dec3=Block(4*ch_mul, 2*ch_mul)
    self.up4=nn.ConvTranspose2d(2*ch_mul, ch_mul, kernel_size=3, stride=2, padding=1, output_padding=1)
    self.dec4=Block(2*ch_mul, ch_mul, seperable=False)

    self.final=nn.Conv2d(ch_mul, squeeze, kernel_size=(1, 1))
    self.softmax = nn.Softmax2d()
      
  def forward(self, x):

    enc1=self.enc1(x)

    enc2=self.enc2(F.max_pool2d(enc1, (2, 2)))

    enc3=self.enc3(F.max_pool2d(enc2, (2,2)))

    enc4=self.enc4(F.max_pool2d(enc3, (2,2)))

    middle=self.middle(F.max_pool2d(enc4, (2,2)))

    up1=torch.cat([enc4, self.up1(middle)], 1)
    dec1=self.dec1(up1)

    up2=torch.cat([enc3, self.up2(dec1)], 1)
    dec2=self.dec2(up2)

    up3=torch.cat([enc2, self.up3(dec2)], 1)
    dec3=self.dec3(up3)

    up4=torch.cat([enc1, self.up4(dec3)], 1)
    dec4=self.dec4(up4)

    final=self.final(dec4)

    return final


class UDec(nn.Module):
  def __init__(self, squeeze, ch_mul=64, in_chans=3):
    super(UDec, self).__init__()

    self.enc1=Block(squeeze, ch_mul, seperable=False)
    self.enc2=Block(ch_mul, 2*ch_mul)
    self.enc3=Block(2*ch_mul, 4*ch_mul)
    self.enc4=Block(4*ch_mul, 8*ch_mul)

    self.middle=Block(8*ch_mul, 16*ch_mul)

    self.up1=nn.ConvTranspose2d(16*ch_mul, 8*ch_mul, kernel_size=3, stride=2, padding=1, output_padding=1)
    self.dec1=Block(16*ch_mul, 8*ch_mul)
    self.up2=nn.ConvTranspose2d(8*ch_mul, 4*ch_mul, kernel_size=3, stride=2, padding=1, output_padding=1)
    self.dec2=Block(8*ch_mul, 4*ch_mul)
    self.up3=nn.ConvTranspose2d(4*ch_mul, 2*ch_mul, kernel_size=3, stride=2, padding=1, output_padding=1)
    self.dec3=Block(4*ch_mul, 2*ch_mul)
    self.up4=nn.ConvTranspose2d(2*ch_mul, ch_mul, kernel_size=3, stride=2, padding=1, output_padding=1)
    self.dec4=Block(2*ch_mul, ch_mul, seperable=False)
    
    self.final=nn.Conv2d(ch_mul, in_chans, kernel_size=(1, 1))
      
  def forward(self, x):
    enc1 = self.enc1(x)

    enc2 = self.enc2(F.max_pool2d(enc1, (2, 2)))

    enc3 = self.enc3(F.max_pool2d(enc2, (2,2)))

    enc4 = self.enc4(F.max_pool2d(enc3, (2,2)))

    middle = self.middle(F.max_pool2d(enc4, (2,2)))

    up1 = torch.cat([enc4, self.up1(middle)], 1)
    dec1 = self.dec1(up1)

    up2 = torch.cat([enc3, self.up2(dec1)], 1)
    dec2 = self.dec2(up2)

    up3 = torch.cat([enc2, self.up3(dec2)], 1)
    dec3 =self.dec3(up3)

    up4 = torch.cat([enc1, self.up4(dec3)], 1)
    dec4 = self.dec4(up4)

    final=self.final(dec4)

    return final


class WNet(nn.Module):
  def __init__(self, squeeze, ch_mul=64, in_chans=3, out_chans=1000):
    super(WNet, self).__init__()
    if out_chans==1000:
        out_chans=in_chans
    self.UEnc=UEnc(squeeze, ch_mul, in_chans)
    self.UDec=UDec(squeeze, ch_mul, out_chans)

  def forward(self, x, returns='both'):
    enc = self.UEnc(x)

    if returns=='enc':
      return enc

    dec = self.UDec(F.softmax(enc, 1))

    if returns=='dec':
      return dec

    if returns=='both':
      return enc, dec
    else:
      raise ValueError('Invalid returns, returns must be in [enc dec both]')

## Wnet train/test/loss

In [ ]:
softmax = nn.Softmax2d()
criterionIdt = torch.nn.MSELoss()

def train_op(model, optimizer, input, k, img_size, psi=0.5):
  enc = model(input, returns='enc')
  d = enc.clone().detach()
  n_cut_loss=soft_n_cut_loss(input,  softmax(enc),  img_size)
  n_cut_loss.backward()
  optimizer.step()
  optimizer.zero_grad()

  dec = model(input, returns='dec')
  rec_loss=reconstruction_loss(input, dec)
  rec_loss.backward()
  optimizer.step()
  optimizer.zero_grad()

  return (model, n_cut_loss, rec_loss)


def reconstruction_loss(x, x_prime):
  rec_loss = criterionIdt(x_prime, x)
  return rec_loss


def test():
  wnet=WNet.WNet(4)
  synthetic_data=torch.rand((1, 3, 128, 128))
  optimizer=torch.optim.SGD(wnet.parameters(), 0.001) #.cuda()
  train_op(wnet, optimizer, synthetic_data)


def show_image(image):
  img = image.numpy().transpose((1, 2, 0))
  plt.imshow(img)
  plt.show()

In [ ]:
epochsall = 100

# Create empty lists for average N_cut losses and reconstruction losses
n_cut_losses_avg = []
rec_losses_avg = []

# Squeeze k
k = 4
img_size =  # define image size from test_loader
wnet = WNet(k).to(device)

learning_rate = 0.003
optimizer = torch.optim.SGD(wnet.parameters(), lr=learning_rate)

transform = transforms.Compose([transforms.Resize(img_size),
                                transforms.ToTensor()])

# Train 1 image set batch size=1 and set shuffle to False
# Here we can train new model for each image, or batch it
dataloader = train_loader  # or test_loader

# Run for every epoch
for epoch in range(epochsall):

  # At 1000 epochs divide SGD learning rate by 10
  if (epoch > 0 and epoch % 1000 == 0):
    learning_rate = learning_rate/10
    optimizer = torch.optim.SGD(wnet.parameters(), lr=learning_rate)

  # Print out every epoch:
  print(f"Epoch = {epoch}")

  # Create empty lists for N_cut losses and reconstruction losses
  n_cut_losses = []
  rec_losses = []
  start_time = time.time()

  for (idx, batch) in enumerate(dataloader):
    # Train 1 image idx > 1
    if(idx > 1): break

    # Train Wnet with CUDA if available
    batch[0] = batch[0].to(device)
    
    wnet, n_cut_loss, rec_loss = train_op(wnet, optimizer, batch[0], k, img_size)

    n_cut_losses.append(n_cut_loss.detach())
    rec_losses.append(rec_loss.detach())

  n_cut_losses_avg.append(torch.mean(torch.FloatTensor(n_cut_losses)))
  rec_losses_avg.append(torch.mean(torch.FloatTensor(rec_losses)))
  print("--- %s seconds ---" % (time.time() - start_time))


images, labels = next(iter(dataloader))

# Run wnet with cuda if enabled
images = images.to(device)

enc, dec = wnet(images)

torch.save(wnet.state_dict(), "model_" + args.name)
wnet_model = wnet
np.save("n_cut_losses_" + args.name, n_cut_losses_avg)
np.save("rec_losses_" + args.name, rec_losses_avg)
print("Done")

## Apply Wnet to set of images

In [ ]:
model = WNet.WNet(4).to(device)  # squezze layers

model.load_state_dict(torch.load(wnet_model))
model.eval()

transform = transforms.Compose([transforms.Resize((64, 64)),
                            transforms.ToTensor()])

image = Image.open(args.image).convert('RGB')
x = transform(image)[None, :, :, :]

enc, dec = model(x)


plt.figure()
show_image(x[0])
plt.show()

plt.figure()
show_image(enc[0, :1, :, :].detach())
plt.show()

plt.figure()
show_image(dec[0, :, :, :].detach())
plt.show()